In [2]:
!pip install -U langchain
# Requires Python 3.10+


[notice] A new release of pip available: 22.2.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install -U langchain-openai


[notice] A new release of pip available: 22.2.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [21]:
!pip install pydantic


[notice] A new release of pip available: 22.2.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="gpt-4o-mini",
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='a471c3ad-454c-4d49-b440-fbf015621114'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 56, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CzkHcuqtC2puNBsYHUpnf56as8xbd', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bd699-b1a2-7641-8899-00541cf6f814-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_g549A9ACpAMRgewF5pcVpjf5', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 56, 'ou

In [5]:
from dataclasses import dataclass

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy

In [6]:
# Define system prompt
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

In [7]:
# Define context schema
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

In [8]:
# Define tools
@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

In [11]:
# Configure model
model = init_chat_model(
    "gpt-4o-mini",
    temperature=0
)

In [9]:
# Define response format
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

In [12]:
# Set up memory
checkpointer = InMemorySaver()

# Create agent
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

In [18]:
# Run agent
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)


In [19]:
print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )

ResponseFormat(punny_response="Looks like Florida is still shining bright with sunny skies! It's a perfect day to let the sunshine in!", weather_conditions='sunny')


In [20]:
# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "but i am staying in singapore tho?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

ResponseFormat(punny_response="Looks like Singapore is also basking in the sunshine! It's a bright day to enjoy the tropical vibes!", weather_conditions='sunny')


In [25]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy, ProviderStrategy


class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model="gpt-4o-mini",
    # tools=[search_tool],
    response_format=ToolStrategy(ContactInfo)
    # response_format=ProviderStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [27]:
for chunk in agent.stream({
    "messages": [{"role": "user", "content": "Search for AI news and summarize the findings"}]
}, stream_mode="values"):
    # Each chunk contains the full state at that point
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    # elif latest_message.tool_calls:
        # print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

Agent: Search for AI news and summarize the findings
Agent: Returning structured response: name='AI News' email='news@aitech.com' phone='123-456-7890'


In [29]:
full = None  # None | AIMessageChunk
for chunk in model.stream("What color is the sky?"):
    full = chunk if full is None else full + chunk
    print(full.text)


The
The color
The color of
The color of the
The color of the sky
The color of the sky can
The color of the sky can vary
The color of the sky can vary depending
The color of the sky can vary depending on
The color of the sky can vary depending on several
The color of the sky can vary depending on several factors
The color of the sky can vary depending on several factors,
The color of the sky can vary depending on several factors, including
The color of the sky can vary depending on several factors, including the
The color of the sky can vary depending on several factors, including the time
The color of the sky can vary depending on several factors, including the time of
The color of the sky can vary depending on several factors, including the time of day
The color of the sky can vary depending on several factors, including the time of day,
The color of the sky can vary depending on several factors, including the time of day, weather
The color of the sky can vary depending on several fa

In [30]:
responses = model.batch([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
])
for response in responses:
    print(response)

content='Parrots have colorful feathers for several reasons, primarily related to their survival, communication, and mating. Here are some key factors:\n\n1. **Camouflage**: In their natural habitats, which often include lush forests and tropical environments, the vibrant colors of parrots can help them blend in with the foliage. This camouflage can protect them from predators.\n\n2. **Communication**: Colorful feathers can play a role in social interactions among parrots. Bright colors can signal health and vitality, which are important in establishing social hierarchies and relationships within flocks.\n\n3. **Mating Displays**: Bright plumage is often a sign of fitness and genetic quality. During mating season, colorful feathers can attract potential mates, as they may indicate that an individual is healthy and has good genes.\n\n4. **Species Identification**: The distinct colors and patterns of feathers help parrots identify each other and differentiate between species. This is cru

In [31]:
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


model_with_tools = model.bind_tools([get_weather])  

response = model_with_tools.invoke("What's the weather like in Boston?")
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

Tool: get_weather
Args: {'location': 'Boston'}


In [33]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie)
response = model_with_structure.invoke("Provide details about the movie Ponniyin Selvan")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

title='Ponniyin Selvan: I' year=2022 director='Mani Ratnam' rating=8.0


In [34]:
for chunk in model.stream("Why do parrots have colorful feathers?"):
    reasoning_steps = [r for r in chunk.content_blocks if r["type"] == "reasoning"]
    print(reasoning_steps if reasoning_steps else chunk.text)


Par
rots
 have
 colorful
 feathers
 for
 several
 reasons
,
 primarily
 related
 to
 communication
,
 camouflage
,
 and
 mate
 attraction
.


1
.
 **
Communication
**
:
 Bright
ly
 colored
 feathers
 can
 help
 parro
ts
 communicate
 with
 each
 other
.
 In
 dense
 forests
 where
 they
 often
 live
,
 vibrant
 colors
 can
 signal
 species
 identity
 and
 social
 status
 to
 other
 parro
ts
.


2
.
 **
Mate
 Attraction
**
:
 Color
ful
 plum
age
 is
 often
 a
 sign
 of
 health
 and
 vitality
.
 During
 mating
 season
,
 brighter
 colors
 can
 attract
 potential
 mates
,
 as
 they
 may
 indicate
 good
 genetics
 and
 the
 ability
 to
 thrive
 in
 their
 environment
.


3
.
 **
Cam
ouflage
**
:
 While
 bright
 colors
 may
 seem
 counter
int
uitive
 for
 camouflage
,
 many
 parro
ts
'
 colors
 can
 help
 them
 blend
 into
 the
 d
apple
d
 light
 of
 their
 forest
 habitats
.
 The
 greens
,
 yell
ows
,
 and
 blues
 can
 mimic
 the
 foliage
 and
 flowers
,
 making
 it
 harder
 for
 predators